# Spark Cluster Slave Nodes termination

### Importing Required Libraries ::
* __boto3__: Required to connect to AWS
* __botocore__: Required to handle the exceptions related to boto3 tasks

In [1]:
import boto3
from botocore.exceptions import ClientError

### Creating boto3 session, clients and resources ::

In [2]:
try:
    session = boto3.session.Session(region_name='us-east-1')
    ec2_client = session.client('ec2')
    ec2_resource = session.resource('ec2')
except ClientError as e:
    print("Unexpected error while creating boto3 session, client and resources: " + str(e))
    exit()

### Declaring the required hardcoded informations ::

In [3]:
user = 'root'

### Extract details of running Slave Nodes for current user ::
* Extracted slave nodes will be terminated one by one.
* If there are any running processes, those will be terminated automatically.

In [4]:
try:
    instances = ec2_resource.instances.filter(
        Filters=[
            {
                'Name': 'instance-state-name',
                'Values': ['running']
            },
            {
                'Name': 'tag:Project',
                'Values': ['SparkCluster']
            },
            {
                'Name': 'tag:User',
                'Values': [user]
            },
            {
                'Name': 'tag:NodeType',
                'Values': ['Slave']
            }
        ]
    )
except ClientError as e:
    print("Unexpected error while looking for already running Master node EC2 instance for user-'" + user + "': " + str(e))
    exit()

### Terminating the EC2 for master node on AWS ::
* __terminate__ API is used under EC2 resource to terminate the extracted EC2 servers(Slave Nodes of out Spark Cluster).

In [5]:
if list(instances):
    try:
        for instance in instances:
            instance.terminate()
            print("Slave node Instance('" + str(instance.id) + "') for User('" + user + "') is pushed to termination. It will be terminated within 2-3 mins.")
            
    except ClientError as e:
        print("Unexpected error while trying to terminate the Slave node EC2 servers for User('" + user + "'): " + str(e))
        exit()
else:
    print("There are no running slave nodes for user-'" + user + "'.")

Slave node Instance('i-0d1b1bbaf19af5f60') for User('root') is pushed to termination. It will be terminated within 2-3 mins.
Slave node Instance('i-0d199f2f7e4a24207') for User('root') is pushed to termination. It will be terminated within 2-3 mins.
